In [1]:
import datetime
import gc
import os
import warnings
warnings.filterwarnings("ignore")
import random
from collections import defaultdict
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import Pool, cv
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score,make_scorer,f1_score
from sklearn.model_selection import KFold, train_test_split

from lightgbm import LGBMClassifier,Dataset
from sklearn.model_selection import GridSearchCV

from dataclasses import dataclass


%matplotlib inline
path_dir=os.getcwd()
path_train='../train_dataset_train.csv'
path_test='../test_dataset_test.csv'
path_subm='../Иннополис/sample_solution.csv'

@dataclass
class Config:
    RS: int = 42
    treshold: float = 0.0
    device='cuda:0'
    num_epochs: int = 120
    batch_size: int = 256
    num_workers: int = 2
        
def get_df(path,train=True,drp=0):
    df=pd.read_csv(path)
    df=df.sort_index(axis=1)
    df.drop(['id','.geo','area'],axis=1,inplace=True)
    if train:
       # df=df.loc[df.crop.isin([0,4])]
        df.reset_index(drop=True,inplace=True) 
        ##    
        drop04=[19,223,541,432,598,469],
        drp=list(set(drop04))
        print(f'droped: {len(drp)}')
        df.drop(drp,axis=0,inplace=True)          
        y=df[['crop']]
        df.drop(['crop'],axis=1,inplace=True)  
    
   # df.rename({c:pd.to_datetime(c.replace('nd_mean_','')) for c in df.columns},axis=1,inplace=True) 
    df[df<=0]=0    
    return (df.reset_index(drop=True),y.crop.reset_index(drop=True).values) if train else df.reset_index(drop=True)

X_train,y_train=get_df(path_train)
X_test=get_df(path_test,False)
df_subm=pd.read_csv(path_subm,index_col='id')

droped: 120


In [2]:
y_train[[1805]]#=0

array([6], dtype=int64)

In [3]:
params={      
    'boosting_type':'gbdt',#'gbdt',#'dart',
    'class_weight':'balanced',   
    'objective': 'multiclass',  
   # 'learning_rate':0.20,
    'random_seed':42,
    #'iteration':700,
    #'max_depth':5,#
   # 'min_data_in_leaf':79,#79,57
   # 'lambda_l2': 3,
}
# params={
#  'class_weight':'balanced',   
#  'objective': 'multiclass',     
#  'feature_fraction': 0.45,
#  'bagging_fraction': 0.7,
#  'bagging_freq': 5,
#  'min_child_samples': 44,
#  'n_estimators': 1600,
#  'learning_rate': 0.16686393584936493,
#  'min_data_in_leaf': 150,
#  'lambda_l1': 0.022838066447584147,
#  'lambda_l2': 0.0002581454840728029,
#  'min_gain_to_split': 2.302100113694354,
#  'max_bin': 230}
model_lightgbm=LGBMClassifier(**params)

In [4]:
avg_res=[]


X_train,y_train=get_df(path_train,train=True)
FOLDS=5
kfolds = KFold(n_splits=FOLDS)

drpt_res=dict()
#t=tqdm(np.round(np.arange(0.1,0.8,0.05),2))
result={'recall':[],'cm':[],'epochs_loss':[]}
splits = kfolds.split(X_train,y_train)    
errors=defaultdict(lambda: defaultdict(lambda: 0))   
for fld,(train_index, valid_index) in enumerate(splits):    
    X_tr, X_val = X_train.values[train_index], X_train.values[valid_index]
    y_tr, y_val = y_train[train_index], y_train[valid_index]    

    model_lightgbm=LGBMClassifier(**params)
    model_lightgbm.fit(X_tr,y_tr)

    y_val_pred=model_lightgbm.predict(X_val)

    recall=recall_score(y_true=y_val,y_pred=y_val_pred,average='macro')
    cm=confusion_matrix(y_true=y_val,y_pred=y_val_pred)
    result['recall'].append(recall)
    result['cm'].append(cm)
avg_res.append([sum(result['recall'])/FOLDS,np.round(sum(result['cm'])/FOLDS,0)])


droped: 120


In [5]:
avg_res#0.9651569930528467

[[0.9614087326169896,
  array([[120.,   0.,   2.,   0.,   2.,   0.,   7.],
         [  0., 139.,   0.,   0.,   1.,   0.,   0.],
         [  1.,   0., 136.,   0.,   2.,   0.,   1.],
         [  0.,   0.,   0., 131.,   0.,   0.,   0.],
         [  6.,   1.,   1.,   0., 123.,   0.,   3.],
         [  0.,   0.,   0.,   0.,   0., 133.,   0.],
         [  6.,   0.,   1.,   0.,   2.,   0., 124.]])]]

In [6]:
X_train,y_train=get_df(path_train)
model_lightgbm.fit(X_train,y_train)

droped: 120


LGBMClassifier(class_weight='balanced', objective='multiclass', random_seed=42)

In [7]:
y_test_pred = model_lightgbm.predict(X_test)

df_subm.crop=y_test_pred
df_subm.head()


,crop
id,
611,3
6417,2
3352,3
4224,1
3102,6


In [10]:
df_subm.to_csv(f'my_submission_LS.csv')

In [11]:
model_lightgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': 'balanced',
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': 'multiclass',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'random_seed': 42}